# Buck Converter Efficiency

This study should follow the equations and match the example from [ROHM - Efficiency of Buck Converter](https://d1d2qsbl8m0m72.cloudfront.net/en/products/databook/applinote/ic/power/switching_regulator/buck_converter_efficiency_app-e.pdf) application note.

## Equations for losses

In [10]:
class buck:
    def p_on_high(v_in, i_out, f_sw, L, r_ds_on_h):
        """ Compute high-side mosfet conduction losses for a syncrhonous rectifier topology """
        di_l = ((v_in - v_out) / (f_sw * L)) * (v_out / v_in)
        i_p = i_out + (di_l/2)
        i_v = i_out - (di_l/2)
        p = (i_out**2 + ((i_p - i_v)**2) / 12 ) * r_ds_on_h * (v_out / v_in)
        return p

    def p_on_high(v_in, i_out, f_sw, L, r_ds_on_l):
        """ Compute high-side mosfet conduction losses for a syncrhonous rectifier topology """
        di_l = ((v_in - v_out) / (f_sw * L)) * (v_out / v_in)
        i_p = i_out + (di_l/2)
        i_v = i_out - (di_l/2)
        p = (i_out**2 + ((i_p - i_v)**2) / 12 ) * r_ds_on_l * (1 -(v_out / v_in))
        return p

    def p_sw(v_in, i_out, f_sw, t_r, t_f):
        """ Compute mosfet switching losses for a syncrhonous rectifier topology """
        p = 0.5 * v_in * i_out * (t_r + t_f) * f_sw
        return p

    def p_diode(v_in, i_rr, t_rr, f_sw):
        """ Compute mosfet's diode body losses for a syncrhonous rectifier topology """
        p = 0.5 * v_inn * i_rr * t_rr * f_sw
        return p

    def p_c_oss(v_in, f_sw, c_ds_h, c_ds_l, c_gd_h, c_gd_l):
        """ Compute mosfet output capacitance losses for a syncrhonous rectifier topology """
        c_oss_l = c_ds_l + c_gd_l
        c_oss_h = c_ds_h + c_gd_h
        p = 0.5 * (c_oss_l + c_oss_h) * v_in**2 * f_sw
        return p

    def p_dead(v_d, i_out, t_d_r, t_d_f, f_sw):
        """ Compute mosfet dead time losses for a syncrhonous rectifier topology """
        p = v_d * i_out * (t_d_r + t_d_f) * f_sw
        return p

    def p_gate(c_gs_h, c_gs_l, v_gs, f_sw):
        """ Compute mosfet gate charge losses for a syncrhonous rectifier topology """
        p = (c_gs_h + c_gs_l) * v_gs**2 * f_sw
        return p

    def p_gate_alt(q_g_h, q_g_l, v_gs, f_sw):
        """ Compute mosfet gate charge losses for a syncrhonous rectifier topology """
        p = (q_g_h + q_g_l) * v_gs * f_sw
        return p

    def p_driver(v_in, i_cc):
        """ Compute mosfet driver losses for a syncrhonous rectifier topology """
        p = v_in * i_cc
        return p

    def p_inductor(v_in, v_out, i_out, i_cc, f_sw, L, DCR):
        """ Compute inductor losses for a syncrhonous rectifier topology """
        di_l = ((v_in - v_out) / (f_sw * L)) * (v_out / v_in)
        i_p = i_out + (di_l/2)
        i_v = i_out - (di_l/2)
        p = i_out**2 + ((i_p - i_v)**2/12) * DCR
        return p

    def p_input_capacitor(v_in, v_out, i_out, esr_c_in):
        """ Compute input capacitor losses for a syncrhonous rectifier topology """
        i_c_in_rms = i_out * ((v_in - v_out) * v_out)**(1/2) / v_in
        p = i_c_in_rms**2 * esr_c_in
        return p

    def p_output_capacitor(v_in, v_out, f_sw, L, esr_c_out):
        """ Compute output capacitor losses for a syncrhonous rectifier topology """
        di_l = ((v_in - v_out) / (f_sw * L)) * (v_out / v_in)
        i_c_out_rms = d_il/(2 * (3)**(1/2))
        p = i_c_out_rms**2 * esr_c_out
        return p


## Example from app note.

### Input Data

In [11]:
v_in = 12
v_out = 5
i_out = 3
r_ds_on_h = 100e-3
r_ds_on_l = 70e-3
L = 4.7e-6
f_sw = 1e6
tr_h = 4e-9
tf_h = 6e-9
tr_l = 2e-9
tf_l = 2e-9
v_d = 0.5
i_rr = 0.3
t_rr = 25e-9
c_ds_h = 40e-12
c_gd_h = 40e-12
c_ds_l = 40e-12
c_gd_l = 40e-12
t_d_r = 30e-9
t_d_f = 30e-9
q_g_h = 1e-9
q_g_l = 1e-9
c_gs_h = 200e-12
c_gs_h = 200e-12
v_gs = 5
i_cc = 1e-3
dcr = 80e-3
esr_c_in = 3e-3
esr_c_out = 1e-3

### Compute and verify

In [24]:
def assertf(value:float, expected_value:float, tol:float=0.001) -> bool:
    assert abs(value - expected_value) <= tol, f"expecting {expected_value}, got {value}."

p_on_high = buck.p_on_high(v_in, i_out, f_sw, L, r_ds_on_h)
assertf(p_on_high, 376e-3)


AssertionError: expecting 0.376, got 0.526872032691738.